In [1]:
!pip3 install dash==1.19.0  
!pip3 install jupyter_dash 
!pip3 install dash dash_core_components dash_html_components
!pip3 install --upgrade plotly
!pip3 install pandas dash

In [2]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import plotly.express as px
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
from dash import no_update

In [3]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id = 'site-dropdown',
                                             options=[{'label': 'All Sites', 'value': 'ALL'},
                                                      {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'}, 
                                                      {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                      {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                      {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'}
                                                      ],
                                              value = 'ALL',
                                              placeholder = "Select a launch site here",
                                              searchable = True),  
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id = 'success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id = 'payload-slider',
                                                min = 0, max = 10000, step = 1000,
                                                marks = {0: '0kg',
                                                         2500: '2500kg',
                                                         5000: '5000kg',
                                                         7500: '7500kg',
                                                         10000: '10000kg'},
                                                value = [min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id = 'success-pie-chart', component_property = 'figure'),
              Input(component_id = 'site-dropdown', component_property = 'value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values = 'class', 
        names = 'Launch Site', 
        title = 'Total successful launches by launch sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df_cc = filtered_df.groupby(['Launch Site', 'class']).size().reset_index(name = 'class count')
        fig = px.pie(filtered_df_cc, values = 'class count', 
        names = 'class', 
        title = f'Total successful launches for launch site {entered_site}')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id = 'success-payload-scatter-chart', component_property = 'figure'),
              [Input(component_id = 'site-dropdown', component_property = 'value'),
              Input(component_id = 'payload-slider', component_property = 'value')])
def get_scatter_chart(entered_site, payload_range):
    filtered_df1 = spacex_df[(spacex_df['Payload Mass (kg)'].between(payload_range[0], payload_range[1]))]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df1, x = 'Payload Mass (kg)', y = 'class', 
                         color = 'Booster Version Category', 
                         title = 'Correlation between payload mass (kg) and success rate for all launch sites')
        return fig
    else:
        filtered_df2 = filtered_df1[filtered_df1['Launch Site'] == entered_site]
        fig = px.scatter(filtered_df2, x = "Payload Mass (kg)", y = "class",  
                         color = 'Booster Version Category',
                         title = f'Correlation between payload mass (kg) and success rate for launch site {entered_site}')
        return fig
        

# Run the app
if __name__ == '__main__':
    app.run_server(host = "127.0.0.1", port = 8050, dev_tools_ui = False, 
                   debug = False, dev_tools_props_check=False)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Jan/2022 07:03:56] "GET /_alive_4ec9dab9-a37e-498f-8dfa-9f2ed966329a HTTP/1.1" 200 -


Dash app running on:


<IPython.core.display.Javascript object>

In [4]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0
